The notebook gives the experiment reported in Fig. 6, where the APIS recovers images of the Set12 dataset that are adversarially corrupted with sparse salt-and-pepper noise.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import cv2
from skimage import data
from numpy.fft import fft2, ifft2
from imglib import HT, obfuscation, noiseSP, rescale
from skimage.restoration import denoise_tv_bregman
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity as ssim
import time
import os

In [2]:
def apis_fourier_cann(noisy_img, lr_img, frac_k, frac_s, max_iter=10):
    shape=noisy_img.shape
    noise = np.zeros(shape)
    loss=np.zeros(max_iter)
    for i in range(max_iter):
        clean_img= abs(ifft2(HT(fft2(noisy_img-noise),frac_s)))
        noise=HT(noisy_img-clean_img, frac_k)
        loss[i]=np.linalg.norm(clean_img-lr_img, ord='fro')
    return rescale(clean_img), loss

In [3]:
noise_name='salt_pepper_noise/'

if not os.path.exists('output/'+str(noise_name)+'apis/'):
        os.makedirs('output/'+str(noise_name)+'apis/')

In [4]:
metrics=np.zeros(shape=(12,11))

for i in range(12):
    img=cv2.imread('input/Set12/'+str(i+1)+'.png',0)
    noisy_img=cv2.imread('input/salt_pepper_noise/'+str(i+1)+'.png',0)
    
    noisy_psnr=peak_signal_noise_ratio(img, noisy_img)
    noisy_ssim = ssim(img, noisy_img )
    
    metrics[i,0], metrics[i,1] = noisy_psnr, noisy_ssim
    print(f'img_no: {i}, noisy psnr={noisy_psnr:.2f}, ssim={noisy_ssim:.2f}')
    
    #---------------APIS------------------#
    frac_k=0.2
    frac_s=0.1
    start_time=time.time()
    apis_img, loss=apis_fourier_cann(noisy_img, img, frac_k, frac_s, max_iter=50)
    apis_time = time.time()-start_time
    apis_psnr=peak_signal_noise_ratio(img, apis_img )
    apis_ssim = ssim(img, apis_img )
    
    metrics[i,2], metrics[i,3], metrics[i,4] = apis_psnr, apis_ssim, apis_time
    print(f'apis psnr={apis_psnr:.2f}, ssim={apis_ssim:.2f}, time={apis_time:.2f}s')
    plt.imsave('output/'+str(noise_name)+'apis/'+str(i+1)+'.png',apis_img, cmap='gray')
    
    

img_no: 0, noisy psnr=9.93, ssim=0.04
apis psnr=27.36, ssim=0.90, time=1.82s
img_no: 1, noisy psnr=10.61, ssim=0.02
apis psnr=32.60, ssim=0.98, time=1.71s
img_no: 2, noisy psnr=9.90, ssim=0.05
apis psnr=29.52, ssim=0.84, time=1.78s
img_no: 3, noisy psnr=9.92, ssim=0.05
apis psnr=36.43, ssim=0.97, time=1.74s
img_no: 4, noisy psnr=10.30, ssim=0.07
apis psnr=25.13, ssim=0.79, time=1.84s
img_no: 5, noisy psnr=9.42, ssim=0.05
apis psnr=26.21, ssim=0.84, time=1.77s
img_no: 6, noisy psnr=9.73, ssim=0.05
apis psnr=35.71, ssim=0.95, time=1.80s
img_no: 7, noisy psnr=10.54, ssim=0.04
apis psnr=30.00, ssim=0.85, time=1.80s
img_no: 8, noisy psnr=10.16, ssim=0.07
apis psnr=26.88, ssim=0.76, time=1.78s
img_no: 9, noisy psnr=10.68, ssim=0.05
apis psnr=28.91, ssim=0.80, time=1.86s
img_no: 10, noisy psnr=10.43, ssim=0.05
apis psnr=24.06, ssim=0.79, time=1.83s
img_no: 11, noisy psnr=10.81, ssim=0.05
apis psnr=27.44, ssim=0.82, time=1.77s
